<a href="https://colab.research.google.com/github/jaothan26/digital_assets_analysis_/blob/main/BitcoinNetworkRatios_from_CoinMetricsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Less than 3 mns to run
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#https://pypi.org/project/coinmetrics-api-client/
#https://coinmetrics.github.io/api-client-python/site/api_client.html
!pip install coinmetrics.api_client --quiet
from coinmetrics.api_client import CoinMetricsClient
client = CoinMetricsClient()
print(client)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
#Retrieve metrics from coinmetrics
metrics = client.get_asset_metrics(assets = 'btc' ,
                                   metrics =['CapMrktCurUSD', 'CapRealUSD', 'TxTfrValAdjUSD', 'PriceUSD',
                            'AdrActCnt', 'CapMVRVCur', 'NVTAdj','VelCur1yr', 'TxCnt', 'SplyCur', 'HashRate'] ,
                             start_time = '2022-03-01',
                             end_time = '2024-11-04',
                             frequency = '1d')

In [ ]:
#Create the df
df = pd.DataFrame(metrics)
df.head()
df['time'] = pd.to_datetime(df['time'])
df['time'] = df['time'].dt.strftime('%Y-%m-%d')
cols_to_convert = df.columns.difference(['time'])  # Select all columns except 'time'
df[cols_to_convert] = df[cols_to_convert].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [ ]:
# Helper function to calculate rolling metrics
def calculate_rolling_metric(df, metric_name, numerator, denominator, windows):
    series_names = []
    for window in windows:
        column_name = f"{metric_name}Roll{window}"
        df[column_name] = df[numerator] / df[denominator].rolling(window).mean()
        series_names.append(column_name)
    return series_names

In [ ]:
# Main function to calculate all metrics
def calculate_metrics(df):
    # NVT Signal Ratio (NVTS)
    df['NVTRatioAdj'] = df['CapMrktCurUSD'] / df['TxTfrValAdjUSD']

    # Metrics to calculate with configuration
    metrics_config = [
        {
            'name': 'NVTRatioAdjRoll',
            'numerator': 'CapMrktCurUSD',
            'denominator': 'TxTfrValAdjUSD',
            'windows': [7, 14, 28, 56, 70]
        },
        {
            'name': 'NVTSRatioRoll',
            'numerator': 'CapMrktCurUSD',
            'denominator': 'TxTfrValAdjUSD',
            'windows': [7, 14, 28, 56, 70, 90, 140]
        },
        {
            'name': 'RVTRatioRoll',
            'numerator': 'CapRealUSD',
            'denominator': 'TxTfrValAdjUSD',
            'windows': [7, 14, 28, 56, 70, 90, 140]
        },
    ]

    # Calculate each metric series based on configuration
    for config in metrics_config:
        calculate_rolling_metric(df, config['name'], config['numerator'], config['denominator'], config['windows'])

    # MVRV Ratio
    df['MVRV'] = df['CapMrktCurUSD'] / df['CapRealUSD']

    # Mayer Multiple
    df['MayerMultiple'] = df['PriceUSD'] / df['PriceUSD'].rolling(200).mean()

    # Mayer Multiple Standard Deviation
    df['MayerMultipleStDev'] = df['MayerMultiple'].rolling(200).std()

    return df

In [ ]:
# Run calculations
data = calculate_metrics(df)
df = data

In [ ]:
#Plot Several metrics per graph
import plotly.graph_objects as go

def plot_multiple_metrics(df, x_series='time', y1_series=None, y2_series=None,
                          title='Metrics Comparison',
                          y1_thresholds=None,
                          y1_axis_title='Metric (Y1)', y2_axis_title='Metric (Y2)',
                          y1_axis_type='linear', y2_axis_type='log'):
    """
    Plot multiple metrics on a dual-axis chart with optional thresholds and customized styling.

    Parameters:
        df: DataFrame containing the data.
        x_series: Column name in `df` for the x-axis.
        y1_series: List of dictionaries for left Y-axis series. Each dictionary should contain:
                   {'column': str, 'name': str, 'color': str}.
        y2_series: List of dictionaries for right Y-axis series. Each dictionary should contain:
                   {'column': str, 'name': str, 'color': str}.
        title: Plot title.
        y1_thresholds: Tuple of (lower, upper) thresholds for Y1 metrics. Applies color bands.
        y1_axis_title: Label for the left Y-axis.
        y2_axis_title: Label for the right Y-axis.
        y1_axis_type: Type for the left Y-axis ('linear' or 'log').
        y2_axis_type: Type for the right Y-axis ('linear' or 'log').
    """

    fig = go.Figure()

    # Plot each series on the left Y-axis
    if y1_series:
        for series in y1_series:
            fig.add_trace(go.Scatter(
                x=df[x_series], y=df[series['column']],
                mode='lines', name=series['name'],
                line=dict(color=series['color'], width=2.5),
                yaxis='y1'
            ))

    # Plot each series on the right Y-axis
    if y2_series:
        for series in y2_series:
            fig.add_trace(go.Scatter(
                x=df[x_series], y=df[series['column']],
                mode='lines', name=series['name'],
                line=dict(color=series['color'], width=2.5, dash='dash'),
                yaxis='y2'
            ))

    # Add threshold zones for Y1-axis
    if y1_thresholds:
        fig.add_hrect(
            y0=y1_thresholds[1], y1=df[y1_series[0]['column']].max(),
            fillcolor="rgba(255, 99, 71, 0.2)", line_width=0, yref="y1",
            annotation_text="Sell Zone", annotation_position="top left",
            annotation=dict(font_size=12, font_color="rgba(255, 99, 71, 0.8)")
        )
        fig.add_hrect(
            y0=df[y1_series[0]['column']].min(), y1=y1_thresholds[0],
            fillcolor="rgba(34, 139, 34, 0.2)", line_width=0, yref="y1",
            annotation_text="Buy Zone", annotation_position="bottom left",
            annotation=dict(font_size=12, font_color="rgba(34, 139, 34, 0.8)")
        )

    # Customize layout
    fig.update_layout(
        title=title,
        xaxis=dict(
            title='Date', tickformat='%Y-%m',
            showgrid=True, gridcolor='rgba(220,220,220,0.5)', color='black'
        ),
        yaxis=dict(
            title=y1_axis_title, type=y1_axis_type, side='left',
            showgrid=True, gridcolor='rgba(200,200,200,0.5)', color='black',
        ),
        yaxis2=dict(
            title=y2_axis_title, type=y2_axis_type, side='right',
            overlaying='y', showgrid=False, color='black'
        ),
        plot_bgcolor='white',
        legend=dict(x=0.5, y=1.15, xanchor='center', orientation='h'),
        hovermode="x unified",
        font=dict(color='black')
    )

    # Show the figure
    fig.show()




In [ ]:
# Example Usage with Multiple Metrics
plot_multiple_metrics(
    df,
    x_series='time',
    y1_series=[
        {'column': 'NVTRatioAdj', 'name': 'NVT Ratio Adjusted', 'color': 'teal'},
        {'column': 'RVTRatioRollRoll7', 'name': 'RVT Ratio (7-day)', 'color': 'blue'}
    ],
    y2_series=[
        {'column': 'PriceUSD', 'name': 'Price USD', 'color': 'gray'}
    ],
    title='NVT and RVT Ratios with Price (Dual-Axis)',
    y1_thresholds=(0.8, 1.5),
    y1_axis_title='NVT/RVT Ratios',
    y2_axis_title='Price USD',
    y1_axis_type='linear',
    y2_axis_type='log'
)


In [ ]:
import plotly.graph_objects as go

def plot_multiple_metrics(df, x_series='time', y1_series=None, y2_series=None,
                          title='Metrics Comparison',
                          y1_thresholds=None,
                          y1_axis_title='Metric (Y1)', y2_axis_title='Metric (Y2)',
                          y1_axis_type='linear', y2_axis_type='log'):
    """
    Plot multiple metrics on a dual-axis chart with optional thresholds and customized styling.
    """

    fig = go.Figure()

    # Plot each series on the left Y-axis
    if y1_series:
        for series in y1_series:
            fig.add_trace(go.Scatter(
                x=df[x_series], y=df[series['column']],
                mode='lines', name=series['name'],
                line=dict(color=series['color'], width=2.5),
                yaxis='y1'
            ))

    # Plot each series on the right Y-axis
    if y2_series:
        for series in y2_series:
            fig.add_trace(go.Scatter(
                x=df[x_series], y=df[series['column']],
                mode='lines', name=series['name'],
                line=dict(color=series['color'], width=2.5, dash='dash'),
                yaxis='y2'
            ))

    # Add threshold zones for Y1-axis
    if y1_thresholds:
        fig.add_hrect(
            y0=y1_thresholds[1], y1=df[y1_series[0]['column']].max(),
            fillcolor="rgba(255, 99, 71, 0.2)", line_width=0, yref="y1",
            annotation_text="Sell Zone", annotation_position="top left",
            annotation=dict(font_size=12, font_color="rgba(255, 99, 71, 0.8)")
        )
        fig.add_hrect(
            y0=df[y1_series[0]['column']].min(), y1=y1_thresholds[0],
            fillcolor="rgba(34, 139, 34, 0.2)", line_width=0, yref="y1",
            annotation_text="Buy Zone", annotation_position="bottom left",
            annotation=dict(font_size=12, font_color="rgba(34, 139, 34, 0.8)")
        )

    # Customize layout
    fig.update_layout(
        title=title,
        xaxis=dict(
            title='Date', tickformat='%Y-%m',
            showgrid=True, gridcolor='rgba(220,220,220,0.5)', color='black'
        ),
        yaxis=dict(
            title=y1_axis_title, type=y1_axis_type, side='left',
            showgrid=True, gridcolor='rgba(200,200,200,0.5)', color='black',
        ),
        yaxis2=dict(
            title=y2_axis_title, type=y2_axis_type, side='right',
            overlaying='y', showgrid=False, color='black'
        ),
        plot_bgcolor='white',
        legend=dict(x=0.5, y=1.15, xanchor='center', orientation='h'),
        hovermode="x unified",
        font=dict(color='black')
    )

    # Show the figure
    fig.show()


In [ ]:
#Plot one graph per metric
def plot_metrics_series(df):
    """
    Iterates through multiple metric pairs and plots each one against 'PriceUSD'.
    """
    # Define metric pairs for plotting
    metric_pairs = [
        {'y1_metric': 'NVTRatioAdjRollRoll70', 'y1_title': 'NVT Ratio Adj Roll (70-day)'},
        {'y1_metric': 'NVTSRatioRollRoll90', 'y1_title': 'NVT Signal Ratio (90-day)'},
        {'y1_metric': 'RVTRatioRollRoll7', 'y1_title': 'RVT Ratio (7-day)'},
        {'y1_metric': 'MVRV', 'y1_title': 'MVRV Ratio'},
        {'y1_metric': 'MayerMultiple', 'y1_title': 'Mayer Multiple'},
        {'y1_metric': 'MayerMultipleStDev', 'y1_title': 'Mayer Multiple Std Dev'}
    ]

    # Loop through each metric pair and plot
    for pair in metric_pairs:
        plot_multiple_metrics(
            df,
            x_series='time',
            y1_series=[{'column': pair['y1_metric'], 'name': pair['y1_title'], 'color': 'teal'}],
            y2_series=[{'column': 'PriceUSD', 'name': 'Price USD', 'color': 'gray'}],
            title=f"{pair['y1_title']} vs Price USD",
            y1_thresholds=(0.8, 1.5),
            y1_axis_title=pair['y1_title'],
            y2_axis_title='Price USD',
            y1_axis_type='linear',
            y2_axis_type='log'
        )


# Execute the function to generate all plots
plot_metrics_series(df)


In [ ]:
#Side by side comparison of metrics (overview)
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_metrics_grid(df):
    """
    Creates a grid of subplots for each metric against 'PriceUSD'.
    """
    # Define metric pairs for plotting with colors
    metric_pairs = [
        {'y1_metric': 'NVTRatioAdjRollRoll70', 'y1_title': 'NVT Ratio Adj Roll (70-day)'},
        {'y1_metric': 'NVTSRatioRollRoll90', 'y1_title': 'NVT Signal Ratio (90-day)'},
        {'y1_metric': 'RVTRatioRollRoll7', 'y1_title': 'RVT Ratio (7-day)'},
        {'y1_metric': 'MVRV', 'y1_title': 'MVRV Ratio'},
        {'y1_metric': 'MayerMultiple', 'y1_title': 'Mayer Multiple'},
        {'y1_metric': 'MayerMultipleStDev', 'y1_title': 'Mayer Multiple Std Dev'}
    ]

    # Define rows and columns for a 2x3 grid
    rows = 3
    cols = 2

    # Create subplots with shared x-axis and specify row/column titles
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=[pair['y1_title'] for pair in metric_pairs],
        shared_xaxes=True,
        vertical_spacing=0.1,
        horizontal_spacing=0.1,
        specs=[[{"secondary_y": True}] * cols] * rows  # Enable secondary Y-axis for each subplot
    )

    # Loop through each metric pair and add it to the appropriate subplot
    for idx, pair in enumerate(metric_pairs):
        row = (idx // cols) + 1
        col = (idx % cols) + 1

        # Default color if 'color' key is missing
        line_color = pair.get('color', 'black')  # Fallback to black if color not found

        # Plot the metric on the left Y-axis
        fig.add_trace(go.Scatter(
            x=df['time'], y=df[pair['y1_metric']],
            mode='lines', name=pair['y1_title'],
            line=dict(color=line_color, width=2.5),
        ), row=row, col=col, secondary_y=False)

        # Plot PriceUSD on the right Y-axis for each subplot
        fig.add_trace(go.Scatter(
            x=df['time'], y=df['PriceUSD'],
            mode='lines', name='Price USD',
            line=dict(color='gray', width=2.5, dash='dash'),
        ), row=row, col=col, secondary_y=True)

        # Add thresholds for sell and buy zones (if applicable)
        if pair['y1_metric'] in ['NVTRatioAdjRollRoll70', 'NVTSRatioRollRoll90', 'RVTRatioRollRoll7']:
            fig.add_hrect(
                y0=1.5, y1=df[pair['y1_metric']].max(),
                fillcolor="rgba(255, 99, 71, 0.2)", line_width=0,
                row=row, col=col, secondary_y=False
            )
            fig.add_hrect(
                y0=df[pair['y1_metric']].min(), y1=0.8,
                fillcolor="rgba(34, 139, 34, 0.2)", line_width=0,
                row=row, col=col, secondary_y=False
            )

    # Update layout for the entire grid figure
    fig.update_layout(
        title="Metrics Comparison Grid",
        xaxis=dict(
            title='Date', tickformat='%Y-%m',
            showgrid=True, gridcolor='rgba(220,220,220,0.5)', color='black'
        ),
        plot_bgcolor='white',
        font=dict(color='black'),
        height=1200,
        showlegend=True,
        legend=dict(x=0.5, y=-0.15, xanchor='center', orientation='h')
    )

    # Customize individual y-axes
    for i, pair in enumerate(metric_pairs, start=1):
        fig['layout'][f'yaxis{i}'].update(title=pair['y1_title'], color=line_color)
        fig['layout'][f'yaxis{i*2}'].update(title="Price USD", color="gray", type="log")

    # Show the entire grid
    fig.show()

# Run the grid plot function
plot_metrics_grid(df)


In [ ]:
#Df cleansing
df["Price Change"] = df["PriceUSD"].pct_change().fillna(0)
df_cleaned_apply = df[df.apply(lambda row: pd.notna(row[["CapMrktCurUSD", "MayerMultipleStDev", "NVTRatioAdj", "Price Change"]]).all(), axis=1)]
df = df_cleaned_apply
df['Date'] = df['time']

<ipython-input-13-54e6e5ed3c05>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_cleaned_apply.T

,398,399,400,401,402,403,404,405,406,407,...,970,971,972,973,974,975,976,977,978,979
asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,2023-04-03,2023-04-04,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,...,2024-10-26,2024-10-27,2024-10-28,2024-10-29,2024-10-30,2024-10-31,2024-11-01,2024-11-02,2024-11-03,2024-11-04
AdrActCnt,985157,1108155,964688,1022873,956362,885739,936112,947327,1146146,1056542,...,727063,654362,756668,831182,794409,789382,819093,690705,627617,811710
CapMVRVCur,1.4091,1.421943,1.42496,1.417262,1.411891,1.413111,1.43278,1.498411,1.524008,1.506773,...,2.038385,2.067245,2.120797,2.199388,2.189505,2.125316,2.096717,2.08847,2.072996,2.042172
CapMrktCurUSD,538531058881.47583,543644520583.245605,545145463891.871094,542442263241.966064,540295858608.647461,540848505456.901672,548489957869.892883,574232641225.834106,585084494471.605469,578651893926.302612,...,1325271090943.350098,1344619989932.58252,1381482823559.958252,1437208187257.821289,1432727864892.05957,1391566605240.438477,1374127913099.304688,1369374466600.230469,1359535717301.938721,1340509299537.117432
CapRealUSD,382180867745.852661,382325094058.347778,382568849463.879517,382739466309.132385,382675240414.551331,382735972861.465698,382815111427.27887,383227822344.697266,383911796529.576416,384033835932.643066,...,650157270228.648438,650440534846.920898,651398078131.075928,653458354446.164917,654361574191.914917,654757377568.578857,655371152181.856079,655683243779.960815,655831302218.558838,656413645761.547974
HashRate,305046344.93138,377232885.493115,323675130.890479,375586794.88083,338032941.785607,349935509.409987,347554997.256287,292803183.441789,366599104.769706,380882187.975726,...,694363719.623232,765702460.145348,798993870.56273,746678795.092981,803749783.401563,751434707.931814,727655130.040615,670584141.732034,627780898.788468,722003987.101051
NVTAdj,141.227638,125.512384,93.298672,120.079377,183.726553,265.931715,307.771145,123.413398,93.961561,80.331428,...,289.829309,359.761808,125.204463,104.910249,110.089775,107.529403,121.41572,312.157267,280.032759,149.258232
PriceUSD,27850.979944,28113.958519,28190.311606,28049.074476,27936.804261,27964.05125,28357.80727,29687.565185,30247.095919,29913.003546,...,67023.339961,68000.147501,69862.520547,72678.78662,72450.284382,70367.081958,69483.58303,69241.679487,68742.754623,67779.09698
SplyCur,19336161.958108,19337174.458075,19338043.208032,19339043.207994,19339930.707955,19340849.457917,19341761.957884,19342530.707841,19343493.207802,19344493.20777,...,19773277.36445,19773780.48945,19774305.489445,19774796.114445,19775324.239434,19775817.989417,19776296.114411,19776736.73941,19777149.239409,19777621.114393


In [ ]:
#LSTM to predict Price from selected features (Network Ratios)
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error


# Select and normalize features
features = df[["CapMrktCurUSD", "MayerMultipleStDev", "NVTRatioAdj", "Price Change"]]


# Select and normalize features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
target = df["PriceUSD"].values.reshape(-1, 1)
scaled_target = scaler.fit_transform(target)

# Split data into train, validation, and test sets (60%, 20%, 20%)
train_size = int(0.6 * len(scaled_features))
val_size = int(0.2 * len(scaled_features))

X_train, X_val, X_test = scaled_features[:train_size], scaled_features[train_size:train_size+val_size], scaled_features[train_size+val_size:]
y_train, y_val, y_test = scaled_target[:train_size], scaled_target[train_size:train_size+val_size], scaled_target[train_size+val_size:]

# Model definition
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Early stopping based on validation loss, with patience of 10 epochs
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Training the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    callbacks=[early_stopping],
    batch_size=16,
    verbose=1
)

# Sequential predictions on the test set
test_predictions = model.predict(X_test)
test_predictions = scaler.inverse_transform(test_predictions).flatten()

# Performance threshold filtering
threshold = 1.5  # 1.5% sensitivity control
filtered_predictions = [
    pred if abs(pred - true) / true * 100 <= threshold else None
    for pred, true in zip(test_predictions, scaler.inverse_transform(y_test).flatten())
]

# Plotting - Actual vs Predicted Prices with on-chain metrics
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plot actual prices
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["PriceUSD"],
        mode="lines",
        name="Actual Price (USD)",
        line=dict(color="blue", width=2),
    ),
    secondary_y=False,
)

# Plot predicted prices over test set period
test_dates = df["Date"][train_size+val_size:]
fig.add_trace(
    go.Scatter(
        x=test_dates,
        y=filtered_predictions,
        mode="lines",
        name="Predicted Price (USD)",
        line=dict(color="orange", dash="dash"),
    ),
    secondary_y=False,
)


# Plot CapMrktCurUSD and MayerMultipleStDev
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["CapMrktCurUSD"],
        mode="lines",
        name="CapMrktCurUSD",
        line=dict(color="green", width=1),
    ),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["MayerMultipleStDev"],
        mode="lines",
        name="MayerMultipleStDev",
        line=dict(color="purple", width=1),
    ),
    secondary_y=True,
)

# Update layout
fig.update_layout(
    title="Ethereum Price Forecast with On-Chain Metrics",
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    yaxis2_title="On-Chain Metrics",
    legend=dict(orientation="h", yanchor="top", y=1.15, xanchor="center", x=0.5),
    font=dict(family="Arial", size=12),
    plot_bgcolor="white"
)

# Set y-axes to log scale for clarity
fig.update_yaxes(type="log", secondary_y=False)
fig.update_yaxes(type="log", secondary_y=True)

fig.show()

# Print final model validation performance
val_mse = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation MSE: {val_mse}")

# Print final model test performance with threshold
test_mse = mean_squared_error(scaler.inverse_transform(y_test), test_predictions)
print(f"Test MSE with threshold: {test_mse}")




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0546 - val_loss: 0.0960
Epoch 2/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0065 - val_loss: 0.0018
Epoch 3/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.5287e-04 - val_loss: 9.2260e-04
Epoch 4/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.6062e-04 - val_loss: 6.1896e-04
Epoch 5/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.4646e-04 - val_loss: 5.5947e-04
Epoch 6/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.3365e-04 - val_loss: 5.5404e-04
Epoch 7/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.9813e-04 - val_loss: 4.7010e-04
Epoch 8/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7243e-04 - val_loss: 4.8995e-04
Epoch 9/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6819e-04 - val_loss: 3.5431e-04
Epoch 10/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3920e-04 - val_loss: 3.3058e-04
Epoch 11/80
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3609e-04 - val_loss: 3.2797e-04
Ep

Validation MSE: 0.00017239709268324077
Test MSE with threshold: 758578.9695892615


In [ ]:
#Test Metrics Calculation

import unittest
import pandas as pd


class TestMetricsCalculations(unittest.TestCase):

    def setUp(self):
        # Define a small sample dataset to use in tests
        '''self.data = pd.DataFrame({
            'CapMrktCurUSD': [100, 150, 200, 250, 300],
            'TxTfrValAdjUSD': [10, 15, 20, 25, 30],
            'CapRealUSD': [80, 120, 160, 200, 240],
            'PriceUSD': [1, 2, 3, 4, 5]
        })'''
        self.data = data
        # Apply the main calculation function
        self.result = calculate_metrics(self.data.copy())

    def test_nvtratioadj_calculation(self):
        # Check if 'NVTRatioAdj' is calculated correctly
        expected_nvtratioadj = self.data['CapMrktCurUSD'] / self.data['TxTfrValAdjUSD']
        pd.testing.assert_series_equal(self.result['NVTRatioAdj'], expected_nvtratioadj, check_names=False)


    def test_nvts_signal_ratio(self):
        # Check if NVTS signal ratios are calculated correctly for each window size
        for window in [7, 14, 28, 56, 70, 90, 140]:
            column_name = f'NVTSRatioRollRoll{window}'
            expected = self.data['CapMrktCurUSD'] / self.data['TxTfrValAdjUSD'].rolling(window).mean()
            pd.testing.assert_series_equal(self.result[column_name], expected, check_names=False)

    def test_rvtratio_calculation(self):
        # Check if RVT Ratios are calculated correctly for each window size
        for window in [7, 14, 28, 56, 70, 90, 140]:
            column_name = f'RVTRatioRollRoll{window}'
            expected = self.data['CapRealUSD'] / self.data['TxTfrValAdjUSD'].rolling(window).mean()
            pd.testing.assert_series_equal(self.result[column_name], expected, check_names=False)

    def test_mvrv_ratio(self):
        # Check if MVRV Ratio is calculated correctly
        expected_mvrv = self.data['CapMrktCurUSD'] / self.data['CapRealUSD']
        pd.testing.assert_series_equal(self.result['MVRV'], expected_mvrv, check_names=False)

    def test_mayer_multiple(self):
        # Check if Mayer Multiple is calculated correctly
        column_name = 'MayerMultiple'
        expected_mayer_multiple = self.data['PriceUSD'] / self.data['PriceUSD'].rolling(200).mean()
        pd.testing.assert_series_equal(self.result[column_name], expected_mayer_multiple, check_names=False)

    def test_mayer_multiple_stdev(self):
        # Check if Mayer Multiple standard deviation is calculated correctly
        column_name = 'MayerMultipleStDev'
        expected_mayer_stdev = self.result['MayerMultiple'].rolling(200).std()
        pd.testing.assert_series_equal(self.result[column_name], expected_mayer_stdev, check_names=False)

    def test_empty_dataframe(self):
        # Check if function can handle an empty DataFrame without errors
        empty_df = pd.DataFrame(columns=self.data.columns)
        result = calculate_metrics(empty_df)
        self.assertTrue(result.empty)

    def test_single_row_dataframe(self):
        # Check function with a single row, expecting NaNs for rolling windows
        single_row_df = pd.DataFrame({
            'CapMrktCurUSD': [100],
            'TxTfrValAdjUSD': [10],
            'CapRealUSD': [80],
            'PriceUSD': [1]
        })
        result = calculate_metrics(single_row_df)
        # All rolling window columns should be NaN
        for column in result.columns:
            if 'Roll' in column:
                self.assertTrue(result[column].isna().all())

# Run the tests in Google Colab
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


........
----------------------------------------------------------------------
Ran 8 tests in 0.172s

OK
